In [32]:
!pip install -qU langchain-groq
!pip install -U langchain-huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
import getpass
import os 

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192", temperature = 0)

 ········


In [35]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(["https://admissions.wisc.edu/apply-as-a-freshman/", "https://admissions.wisc.edu/apply-as-a-transfer/", 
                        "https://admissions.wisc.edu/apply-as-a-reentry-student/", 
                        "https://admissions.wisc.edu/apply-as-a-second-degree-student/"]
)

docs = loader.load()
len(docs[0].page_content)

#docs


21182

In [36]:
len(docs)
print(docs[1].metadata)

{'source': 'https://admissions.wisc.edu/apply-as-a-transfer/', 'title': 'Apply as a Transfer Student – Office of Admissions and Recruitment – UW–Madison', 'description': 'Every year, we are fortunate to receive thousands of\xa0applications\xa0from a diverse range of students who are incredibly bright, engaged, and passionate. They have challenged themselves and those around them to make a difference in the world. They know that the University of Wisconsin–Madison is their next step toward something extraordinary—a place where they can lead.', 'language': 'en-US'}


In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=300)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [38]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [43]:
from langchain_core.prompts import PromptTemplate

template = """You are an addmissions advisor at UW Madison, use the following pieces of context to aid your answer to the question at the end.
If you don't know the answer, say that you don't know, but you recommend the student to consult wisc.edu and reach out to admissions or help offices.
Use three sentences maximum and keep the answer as concise as possible.
If the student has not specified what type of student they are, assume First-Year Student. 
Always say "thanks for asking!" at the end of the answer.

Context: {context}

Question: {question}

Helpful Answer:"""
college_advisor = PromptTemplate.from_template(template)

In [44]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | college_advisor
    | llm
    | StrOutputParser()
)

#rag_chain.invoke("What is RAG?")

In [45]:
# from langchain.chains import create_retrieval_chain
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains.combine_documents import create_stuff_documents_chain

# system_prompt = (
# """You are an addmissions advisor at UW Madison, use the following pieces of context to aid your answer to the question at the end.
# If you don't know the answer, say that you don't know, but you recommend the student to consult wisc.edu and reach out to admissions or help offices.
# Use three sentences maximum and keep the answer as concise as possible.
# If the student has not specified what type of student they are, assume First-Year Student. 
# Always say "thanks for asking!" at the end of the answer.

# Context: {context}"""
# )

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )

# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [46]:
rag_chain.invoke("What are the Math requirnment for incoming freshman?")

'The math requirement for incoming freshmen at UW-Madison is 3 years of math, which includes Algebra and Plane Geometry, as well as College-Preparatory Math, which can be fulfilled with one year in high school or one college course at the level of Algebra 2 or beyond. Thanks for asking!'

In [47]:
rag_chain.invoke("What English tests does UW Madison Accept for international students?")

'UW-Madison accepts English proficiency exams such as TOEFL, IELTS, and Duolingo English Test for international students. However, if you believe you qualify for a waiver based on your high school transcript, please submit the required transcripts to our office for review. Thanks for asking!'

In [48]:
rag_chain.invoke("Is Duoloingo accepted?")

'Yes, the Duolingo English Test (DET) is accepted as an English proficiency exam, with a minimum required score of 115+. Thanks for asking!'

In [49]:
rag_chain.invoke("early action deadline?")

'The Early Action deadline for Fall admission is November 1. Applications and all required materials must be submitted by 11:59 p.m. Pacific Time on this date. Thanks for asking!'

In [50]:
rag_chain.invoke("I missed the early application deadline, what are my options? Can I still reach out to someone?")

'You can still apply through the Fall Regular Decision process, with a deadline of January 15. Make sure to submit your application and all required materials by 11:59 p.m. Pacific Time on January 15. If you have any questions or concerns, I recommend reaching out to our admissions office for guidance. Thanks for asking!'

In [51]:
rag_chain.invoke("What are my limitations if I am a second degree student?")

"As a second degree student, not all programs are able to accept second degree students, and you'll need to review school and college specific requirements to determine if the program you're interested in accepts second degree applicants. Additionally, some colleges like the Wisconsin School of Business and College of Engineering do not accept second degree applicants. Thanks for asking!"